In [8]:
import logging
import os
import tempfile
from abc import ABC, abstractmethod
from copy import deepcopy
from pathlib import Path
from typing import Type

import numpy as np
import pandas as pd
from ConfigSpace import Configuration, ConfigurationSpace
from smac import AlgorithmConfigurationFacade, Scenario
from smac.runhistory.dataclasses import TrialValue

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver

pd.options.display.max_columns = 999

In [2]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=5,
    seed=0,
)

In [4]:
features = train_instances[0].get_features()

In [ ]:
t_c = 100
t_v = 100
K = 2
n = 2
solver_class = TSP_LKH_Solver

experiment = GlobalExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    solver_class=solver_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
remaining_time = np.ones(shape=(K, )) * np.inf
best_portfolio.evaluate(test_instances, remaining_time, "test")

In [ ]:
# t_c = 100
# t_v = 100
# K = 2
# n = 2
# solver_class = TSP_LKH_Solver

# experiment = ParhydraExperiment(
#     t_c=t_c,
#     t_v=t_v,
#     K=K,
#     n=n,
#     solver_class=solver_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [10]:
x = pd.read_csv("test_features.csv").iloc[0]
x
# x[:] = 0
# x.to_dict()

num_nodes                    800.000000
cost_matrix_avg           486713.687336
cost_matrix_std                0.571052
cost_matrix_skew               6.582175
stdTime                        0.050000
mst_length                     0.043974
mst_length_avg                 0.044029
mst_length_std                 0.690201
mst_length_skew                1.118514
mst_degree_avg                 1.997500
mst_degree_std                 0.340460
mst_degree_skew                4.593742
mstTime                        0.030000
cluster_distance_avg           0.100851
cluster_distance_std           0.330410
cluster_distance_skew         -0.860030
clusterTime                    1.200000
tour_const_heu_avg             0.058341
tour_const_std                 0.000000
tour_const_skew                0.000000
ls_impov_per_step_avg          0.006229
ls_impov_per_step_std          0.000389
ls_impov_per_step_skew        -2.449490
ls_steps_2lm_avg               0.121250
ls_steps_2lm_std               0.000000


In [19]:
pd.read_csv("test_features.csv").iloc[0].to_dict()

{'num_nodes': 800.0,
 'cost_matrix_avg': 486713.687335701,
 'cost_matrix_std': 0.571052194,
 'cost_matrix_skew': 6.582175107,
 'stdTime': 0.05,
 'mst_length': 0.043973673,
 'mst_length_avg': 0.044028709,
 'mst_length_std': 0.690201489,
 'mst_length_skew': 1.118514344,
 'mst_degree_avg': 1.9975,
 'mst_degree_std': 0.34046004,
 'mst_degree_skew': 4.593742012,
 'mstTime': 0.03,
 'cluster_distance_avg': 0.100851486,
 'cluster_distance_std': 0.330409984,
 'cluster_distance_skew': -0.86003005,
 'clusterTime': 1.2,
 'tour_const_heu_avg': 0.058340503,
 'tour_const_std': 0.0,
 'tour_const_skew': 0.0,
 'ls_impov_per_step_avg': 0.006229412,
 'ls_impov_per_step_std': 0.000389459,
 'ls_impov_per_step_skew': -2.449489743,
 'ls_steps_2lm_avg': 0.12125,
 'ls_steps_2lm_std': 0.0,
 'ls_steps_2lm_skew': -0.0,
 'ls_maxdist_avg': 0.175,
 'ls_maxdist_std': 0.006123724,
 'ls_maxdist_skew': -2.449489743,
 'ls_bestsol_avg': 39.527347803,
 'ls_bestsol_std': 0.02049119,
 'ls_bestsol_skew': 2.44953108,
 'ls_backb

In [ ]:
from src.database import db_connect
conn = db_connect()

In [3]:
import sqlite3
conn = sqlite3.connect("database/2024_11_12_23_10_13.db")

In [4]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
# df_solvers

In [5]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp
1,data/TSP/CEPS_benchmark/grid/04.tsp
2,data/TSP/CEPS_benchmark/grid/33.tsp
3,data/TSP/CEPS_benchmark/rotation/45.tsp
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp
...,...
495,data/TSP/CEPS_benchmark/implosion/23.tsp
496,data/TSP/CEPS_benchmark/expansion/42.tsp
497,data/TSP/CEPS_benchmark/compression/17.tsp
498,data/TSP/CEPS_benchmark/cluster/47.tsp


In [6]:
df = pd.read_sql_query("SELECT * FROM results", conn)
# df.to_excel("tmp.xlsx")

In [20]:
df.loc[df["comment"] == "configuration"]

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,2.49,2.49,configuration,2024-11-12 22:10:16
1,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,2.72,2.72,configuration,2024-11-12 22:10:16
2,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,2.41,2.41,configuration,2024-11-12 22:10:16
3,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,2.42,2.42,configuration,2024-11-12 22:10:16
4,data/TSP/CEPS_benchmark/grid/04.tsp,6xJZWF2RoHDF8qv96yvmmtIvc2yNAsBPXrATK8KbGv4=,1.52,1.52,configuration,2024-11-12 22:10:16
...,...,...,...,...,...,...
1590,data/TSP/CEPS_benchmark/implosion/29.tsp,-fJeHsOaxEncLrlx4nqbZ4lFoe1PIgw1QsgxWMhuSx8=,100.00,10.00,configuration,2024-11-12 22:56:04
1591,data/TSP/CEPS_benchmark/uniform_portgen/18.tsp,-fJeHsOaxEncLrlx4nqbZ4lFoe1PIgw1QsgxWMhuSx8=,100.00,10.00,configuration,2024-11-12 22:56:04
1592,data/TSP/CEPS_benchmark/linearprojection/05.tsp,-fJeHsOaxEncLrlx4nqbZ4lFoe1PIgw1QsgxWMhuSx8=,100.00,10.00,configuration,2024-11-12 22:56:04
1593,data/TSP/CEPS_benchmark/expansion/04.tsp,-fJeHsOaxEncLrlx4nqbZ4lFoe1PIgw1QsgxWMhuSx8=,100.00,10.00,configuration,2024-11-12 22:56:04


In [7]:
_ = df.loc[df["comment"] == "test"].pivot_table(index="instance_id", columns="solver_id", values="cost")
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_

solver_id,2E8WBbr3nGedyhSDRnfVkf5k3xBfZYC3RJL5SlBES8s=,2S7Zr4bn8HZ1WsVPQc7Nwe1jywwc1i7yyocPCoZsw3I=,OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM=,jeKyQpuuR_KDuRW-AqtJXqJlSl1CFozTT7ZJrOBEm8g=,best,min
instance_id,,,,,,
data/TSP/CEPS_benchmark/cluster/00.tsp,100.0,100.00,100.00,100.00,0,100.00
data/TSP/CEPS_benchmark/cluster/01.tsp,100.0,8.57,0.75,100.00,2,0.75
data/TSP/CEPS_benchmark/cluster/02.tsp,100.0,100.00,6.57,100.00,2,6.57
data/TSP/CEPS_benchmark/cluster/03.tsp,100.0,2.76,3.48,100.00,1,2.76
data/TSP/CEPS_benchmark/cluster/04.tsp,100.0,100.00,100.00,100.00,0,100.00
...,...,...,...,...,...,...
data/TSP/CEPS_benchmark/uniform_portgen/45.tsp,100.0,1.90,0.27,100.00,2,0.27
data/TSP/CEPS_benchmark/uniform_portgen/46.tsp,100.0,1.56,7.53,7.17,1,1.56
data/TSP/CEPS_benchmark/uniform_portgen/47.tsp,100.0,6.67,0.38,100.00,2,0.38


In [13]:
_["min"].describe()

count    470.000000
mean      17.938787
std       35.851337
min        0.100000
25%        0.847500
50%        2.135000
75%        5.267500
max      100.000000
Name: min, dtype: float64

In [14]:
(_["min"] > 90).value_counts()

False    395
True      75
Name: min, dtype: int64

In [15]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

2    257
1    129
3      9
Name: best, dtype: int64

In [11]:
_.sort_values(by="min")

solver_id,2E8WBbr3nGedyhSDRnfVkf5k3xBfZYC3RJL5SlBES8s=,2S7Zr4bn8HZ1WsVPQc7Nwe1jywwc1i7yyocPCoZsw3I=,OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM=,jeKyQpuuR_KDuRW-AqtJXqJlSl1CFozTT7ZJrOBEm8g=,best,min
instance_id,,,,,,
data/TSP/CEPS_benchmark/cluster_netgen/18.tsp,100.0,4.96,0.10,100.0,2,0.10
data/TSP/CEPS_benchmark/linearprojection/36.tsp,100.0,3.97,0.10,100.0,2,0.10
data/TSP/CEPS_benchmark/cluster/30.tsp,100.0,7.06,0.13,9.1,2,0.13
data/TSP/CEPS_benchmark/rotation/34.tsp,100.0,100.00,0.16,100.0,2,0.16
data/TSP/CEPS_benchmark/expansion/45.tsp,100.0,100.00,0.18,100.0,2,0.18
...,...,...,...,...,...,...
data/TSP/CEPS_benchmark/explosion/30.tsp,100.0,100.00,100.00,100.0,0,100.00
data/TSP/CEPS_benchmark/explosion/29.tsp,100.0,100.00,100.00,100.0,0,100.00
data/TSP/CEPS_benchmark/explosion/28.tsp,100.0,100.00,100.00,100.0,0,100.00


In [12]:
config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
solver = TSP_LKH_Solver(config=config)

instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

solver.solve(instance)

PARAMETER_FILE = C:\Users\grzegorzzakrzewski\Documents\DataScience\raw-algorithm-portfolios\temp\config_1120.par
PROBLEM_FILE = data/TSP/CEPS_benchmark/cluster_netgen/18.tsp
Successes/Runs = 1/1
Cost.min = 10555222, Cost.avg = 10555222.00, Cost.max = 10555222
Gap.min = 0.0000%, Gap.avg = 0.0000%, Gap.max = 0.0000%
Trials.min = 7, Trials.avg = 7.0, Trials.max = 7
Time.min = 0.10 sec., Time.avg = 0.10 sec., Time.max = 0.10 sec.
Time.total = 0.12 sec.


(0.12, 0.12)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index